In [ ]:
try:
	__import__('lmdeploy')
	__import__('fastapi')
except ImportError:
	%%capture
	%pip install lmdeploy fastapi

In [ ]:
import os
from lmdeploy import pipeline, TurbomindEngineConfig

from pydantic import BaseModel
from fastapi import FastAPI
from starlette.responses import JSONResponse
from dotenv import load_dotenv
from typing import List

load_dotenv()
small_model = os.getenv('FINETUNE_MODEL_NAME')

In [ ]:
# reference: https://github.com/InternLM/lmdeploy/blob/main/docs/en/inference/pipeline.md
pipe = pipeline(model_path=small_model, model_name=small_model, backend_config=TurbomindEngineConfig(tp=2))

# response = pipe(['Hi, please introduce yourself', 'USA is'])
# print(response)

Hosting with FastAPI

In [ ]:
class Message(BaseModel):
    role: str
    content: str

class MessageList(BaseModel):
    messages: List[Message]


app = FastAPI()


@app.post("/v1/chat/completions")
async def predict(message_list: MessageList):
    response = [pipe([message.content]) for message in message_list.messages]
    return JSONResponse(content=response)

if __name__ == "__main__":
	import uvicorn
	uvicorn.run(app, host='0.0.0.0', port=8000)

In [ ]:
'''
curl http://localhost:8000/v1/chat/completions \
	-H "Content-Type: application/json" \
	-d '{
		"messages": [{"role": "user", "content": "Say this is a test!"}]
	}'
'''